Hello, this seems like a fun task so I thought it would good to write up a simple transfer learning model to see the results. I tend to write my notebooks as a tutorial format for those who are unfamiliar with tensorflow v2 and visual recognition. Feel free to skip all of the comments if it is annoying!

The following works were referenced when creating this tutorial:
* https://www.kaggle.com/drcapa/human-trafficking-2021-starter 

* https://www.kaggle.com/shanmukh05/combat-human-trafficking-2k21-tpu-training 

* https://www.kaggle.com/ateplyuk/human-trafficking-2021-baseline 



All mistakes are self-inflicted; please notify me if any mistakes are spotted.



**Loading in Libraries**

In [ ]:
import tensorflow as tf
from kaggle_datasets import KaggleDatasets
import numpy as np
import pandas as pd
import os
from sklearn import preprocessing


First things first, lets initalize the kaggle's accelerators units for faster computation. This is done by changing the notebook settings in the top right. You will find the TPU and GPU setting under Hardware accelerator. Unfortunately, for this task, you will have to first train the model using TPU, and then submit the best model that you found.

In [ ]:

def intialize_accel(hardware):
    """
    input:
    str: GPU or TPU for hardware accelerator
    
    output:
    strategy -- used later for model definition and fitting
    """
    if hardware =='TPU':
        try:
            tpu = tf.distribute.cluster_resolver.TPUClusterResolver().connect()  
            strategy = tf.distribute.experimental.TPUStrategy(tpu)

            print('TPU Initialized')
            print("TPU Units:", strategy.num_replicas_in_sync)
            return strategy
        except:
            print('TPU Initialization Failed')
    
    
    elif hardware == 'GPU':
        print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
        strategy = tf.distribute.MirroredStrategy()
        return strategy
        
strategy = intialize_accel('TPU')
AUTO = tf.data.experimental.AUTOTUNE


Okay, now that the accelerator units are initalized: lets start exploring our data

In [ ]:
#Setting up Pathways
#DIR = '../input/hotel-id-2021-fgvc8/'
DIR = KaggleDatasets().get_gcs_path()

Train_PATH = DIR + "/train_images/"
Test_PATH = DIR + "/test_images/"

##Loading in Training csv
train_df = pd.read_csv("../input/hotel-id-2021-fgvc8/train.csv")
#dropping duplicates
train_df = train_df.drop_duplicates(subset=['image'])

#checking data properties
print("Number of unique hotel chains: ",train_df.chain.nunique())
print("Number of unique hotels: ",train_df.hotel_id.nunique())
print("Number of Training Samples: ", train_df.shape[0])

print(train_df.head())

Great! The hotel id is what we want, so our final model should output ~7000 classes. Now let us define our parameters. There are two types; data-specific parameters and training parameters. Feel free to play around with the training parameters.

* Class: Number of classes
* Channels: Number of color channesl
* Batch_size: Size of the batches
* size: Size of the final processed image
* Split: Size of training vs validation split

In [ ]:
#Specific to data
Classes =  train_df.hotel_id.nunique()
Channels = 3

##Specific to the model
Batch_size = 8 * strategy.num_replicas_in_sync
size = (200,200)
Split = int(0.9*train_df.shape[0])

Now lets start processing our data. First, let us change the encoding of our hotel ids so that they are lined with each other. This can simply be done using the sklearn preprocessing label encoder. 

In [ ]:
le = preprocessing.LabelEncoder()
train_df['label'] = le.fit_transform(train_df['hotel_id'])
print(train_df[['hotel_id','label']])

We are then going to process our input data. To do this in keras, we are going to formulate the data into datasets and pass the datasets directly to our model. It is also here where we define our data augmentation.

In [ ]:
def image_proces(Path, labels):
    """
    inputs: Tensorflow Dataset that contains the following two properties
        Path: Paths to images
        labels: image labels
        
    output: Tensorflow Dataset that contains
        data: processed images
        labels: image labels
    
    Function:
    Takes a tensorflow dataset of image paths and decodes the images into numpy arrays. The images are then resized.
    """
    data = tf.io.read_file(Path)
    data = tf.image.decode_jpeg(data, channels=3)
    data = tf.image.resize(data, size)
    #data = tf.image.per_image_standardization(data)
    return data,labels

def import_image(Paths,labels):
    """
    inputs:
    Paths: a list of paths to the images
    Y: a list of labels (hotel ids)
    
    outputs:
    dataset: a tensorflow datset that contains images which are decoded and resized
    
    Function: 
    Takes a list of paths and returns a formatted tensorflow dataset
    """
    dataset = tf.data.Dataset.from_tensor_slices((Paths,labels))
    dataset = dataset.map(image_proces,num_parallel_calls=AUTO)
    return dataset

def data_augment(image, labels):
    """
    inputs: Tensorflow Dataset that contains the following two properties
        Images: Images arrays
        labels: image labels
        
    output: Tensorflow Dataset that contains
        Image: processed images
        labels: image labels
    
    Function:
    Takes a tensorflow dataset of image arrays and augments the brightness and contrast.
    """
    #image = tf.image.random_flip_left_right(image)
    #image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image,lower=0.8,upper=1.2)
    return image, labels


Paths = Train_PATH + train_df['chain'].astype(str) + '/' + train_df['image']
Paths_train  = Paths[:Split]
Paths_valid  = Paths[Split:]
Paths_Test = tf.io.gfile.glob(Test_PATH + '*.jpg')


train_dataset= import_image(Paths_train,train_df['label'][:Split])


val_dataset = import_image(Paths_valid,train_df['label'][Split:])
dataset_Test = import_image(Paths_Test,np.arange(len(Paths_Test)))



A quick sanity check: 90% training, 10% validation samples. The input shape matches our intended value.

In [ ]:
print("Number of Validation Samples:",val_dataset.cardinality().numpy())
print("Number of Train Samples:",train_dataset.cardinality().numpy())
print("Number of Test Samples:",dataset_Test.cardinality().numpy())

inspect= list(train_dataset.take(1).as_numpy_iterator())
print('Input Shape:',inspect[0][0].shape, 'Example Label:',inspect[0][1])

Here we are going to define our model. We will be using a pretrained image recognition network (ResNet50) to recognize important features and then train a neural network on top to recognize the hotel. Rather than training the whole thing, we will be fixing the resnet (for now) and training the classification network on top. Since we have $\sim$ 7000 classes, we would need to first perform feature extraction before classifcation, or else our model would be far too large ($\sim$9 million parameters) to train properly. 

In [ ]:
input_shape=[200,200,Channels]
Base = tf.keras.applications.ResNet50(include_top=False,input_shape=input_shape)
Base.trainable = False
#Base.summary()


def create_model(Base,input_shape):
    inputs = tf.keras.Input(shape=(input_shape))
    norm =  tf.keras.layers.experimental.preprocessing.Normalization()
    x = norm(inputs)
    x = Base(x,training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(50,activation="relu", dtype='float32')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    outputs = tf.keras.layers.Dense(Classes,activation="softmax", dtype='float32')(x)
        
    model = tf.keras.Model(inputs, outputs)
    return model


model = create_model(Base,input_shape)

model.summary()

Now we define our compiler with ADAM optimizer. Since we didn't encode our labels into one hot representations, we will be using the sparse categorical cross entropy. Additionally, since I am using the TPU accelerator, we can process multiple step in parallel. This is controlled by the steps_per_execution hyperparameter

In [ ]:
import tensorflow_addons as tfa

def compile_model(model, lr):
    
    optimizer = tf.keras.optimizers.Adam(lr=lr)
    
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics,steps_per_execution=8)
    

    return model

Final steps of data processing: all we are doing here is preparing our training and validation datasets for training. 

* Cache: Stores the dataset after first pass (This might result in memory issues!)
* batch: batches the data up
* shuffle: shuffles the training data
* prefetch: prefetches the next batch during training


In [ ]:
EPOCHS= 1
VERBOSE =1
Train_steps  = Split//Batch_size

train_dataset = train_dataset.cache()
train_dataset = train_dataset.map(data_augment,num_parallel_calls=AUTO)
train_dataset = train_dataset.shuffle(2048).batch(Batch_size)
train_dataset = train_dataset.prefetch(AUTO)

val_dataset = val_dataset.cache()
val_dataset = val_dataset.batch(Batch_size)
val_dataset = val_dataset.prefetch(AUTO)

Setting up callbacks which will runned at the end of every epoch of training.

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1,
                              patience=3, mode='max', min_delta=0.0001,verbose=1)

checkpoint_filepath = './best_model.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                            filepath=checkpoint_filepath,
                            save_weights_only=True,
                            monitor='val_accuracy',
                            mode='max',
                            save_best_only=True,verbose=1)


callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, mode='max', min_delta=0.0001,verbose=1)

Now let's start training!

In [ ]:

with strategy.scope():
    Base = tf.keras.applications.ResNet50(include_top=False,input_shape=input_shape)
    Base.trainable = False
    model = create_model(Base,input_shape)
    model = compile_model(model, lr=0.001)
   

print('Fitting') 

History = model.fit(train_dataset, 
                epochs=EPOCHS,
                callbacks=[reduce_lr,model_checkpoint_callback,callback],
                validation_data = val_dataset,  
                verbose=VERBOSE
               )

In [ ]:
from matplotlib import pyplot as plt
plt.figure(1)
plt.plot(History.history['accuracy'][1:],label='Train')
plt.plot(History.history['val_accuracy'][1:],label='Valid')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure(2)
plt.plot(History.history['loss'][1:],label='Train')
plt.plot(History.history['val_loss'][1:],label='Valid')
plt.xlabel('Epoch')
plt.title('Loss')
plt.legend()
plt.show()

Now let's load up the best model and see how it performs on the test set.

In [ ]:
best_model = create_model(Base,input_shape)
best_model.load_weights(checkpoint_filepath)


In [ ]:
dataset_Test = dataset_Test.batch(3)
predictions = best_model.predict(dataset_Test,verbose=1)


In [ ]:
images_names= [i.split('/')[-1] for i in Paths_Test]
hotels = le.inverse_transform(np.argmax(predictions,axis=1))
submission = pd.DataFrame(list(zip(images_names,hotels)), columns=['image','hotel_id'])

In [ ]:
submission.head()

In [ ]:
submission.to_csv("submission.csv", index=False)